In [1]:
import os
import re
import itertools
import pandas as pd
from sklearn.metrics import cohen_kappa_score as cks
from tabulate import tabulate

### Building the dataframe with the results

In [4]:
datapath = '..\\data\\img_labeling_4th_round\\'

In [5]:
label_files = sorted(os.listdir(datapath))
label_files = [f for f in label_files if 'labels_round4' in f]
for f in label_files:
    print(f)

labels_round4_A001_2021-04-21-08-24-53.csv
labels_round4_A002_2021-04-09-11-39-30.csv
labels_round4_A004_2021-04-21-10-33-20.csv
labels_round4_A005_2021-04-08-03-25-31.csv
labels_round4_A006_2021-04-08-03-25-31.csv


In [6]:
list_dfs = []
for label_file in label_files:
    task = int(re.findall(r'[1-4]{1}', label_file)[0])
    user = re.findall(r'A001|A002|A004|A005|A006', label_file)[0]
    date = re.findall(r'202[0-1]{1}.[0-9]{2}.[0-9]{2}', label_file)[0]
    #date = f'{date}.2021'
    
    df = pd.read_csv(f'{datapath}{label_file}', names=['image name', 'class','w','h'])
    df['task'] = task
    df['user'] = user
    df['date'] = date
    df['date'] = pd.to_datetime(df['date'])
    df.drop(['w','h'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df[['task', 'user', 'date', 'image name', 'class']]
    print(f'Task: {task}, User: {user}, Images Classified: {len(df)}, Date: {date}')
    list_dfs.append(df)

Task: 4, User: A001, Images Classified: 1079, Date: 2021-04-21
Task: 4, User: A002, Images Classified: 1079, Date: 2021-04-09
Task: 4, User: A004, Images Classified: 1079, Date: 2021-04-21
Task: 4, User: A005, Images Classified: 1079, Date: 2021-04-08
Task: 4, User: A006, Images Classified: 1079, Date: 2021-04-08


In [7]:
df_labeling = pd.concat(list_dfs)
df_labeling['id_image'] = pd.Categorical(df_labeling['image name']).codes
df_labeling["class"] = df_labeling["class"].apply(lambda x:x[1:-1])
df_labeling = df_labeling[['task', 'user', 'date', 'image name', 'id_image', 'class']]

In [8]:
df_labeling.head()

,task,user,date,image name,id_image,class
0,4,A001,2021-04-21,1.jpg,0,Non-appealing
1,4,A001,2021-04-21,2.jpg,192,Non-appealing
2,4,A001,2021-04-21,3.jpg,303,Non-appealing
3,4,A001,2021-04-21,4.jpg,414,Appealing
4,4,A001,2021-04-21,5.jpg,525,Appealing


In [9]:
df_labeling['class'].unique()

array(['Non-appealing', 'Appealing'], dtype=object)

In [10]:
df_labeling['image_appearances'] = df_labeling.groupby('id_image')['id_image'].transform('count')
df_labeling = df_labeling[df_labeling['image_appearances'] != 1] 

In [11]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5395 entries, 0 to 1078
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   task               5395 non-null   int64         
 1   user               5395 non-null   object        
 2   date               5395 non-null   datetime64[ns]
 3   image name         5395 non-null   object        
 4   id_image           5395 non-null   int16         
 5   class              5395 non-null   object        
 6   image_appearances  5395 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 305.6+ KB


In [12]:
df_labeling.head()

,task,user,date,image name,id_image,class,image_appearances
0,4,A001,2021-04-21,1.jpg,0,Non-appealing,5
1,4,A001,2021-04-21,2.jpg,192,Non-appealing,5
2,4,A001,2021-04-21,3.jpg,303,Non-appealing,5
3,4,A001,2021-04-21,4.jpg,414,Appealing,5
4,4,A001,2021-04-21,5.jpg,525,Appealing,5


#### Checking the numbers of images classified after fixing names:

In [13]:
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        size = len(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image'].unique())
        print(f'Task: {task}, User: {user}, Images Classified: {size}')
    print()

Task: 4, User: A001, Images Classified: 1079
Task: 4, User: A002, Images Classified: 1079
Task: 4, User: A004, Images Classified: 1079
Task: 4, User: A005, Images Classified: 1079
Task: 4, User: A006, Images Classified: 1079



#### Checking which images were not classified:

In [14]:
images = set(df_labeling.id_image.unique())
dfs = []
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        id_list = []
        ids = images.difference(set(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image']))
        if len(ids) > 0:
            #print(f'Missing files for task: {task}, by user: {user}')
            for _id in ids:
                try:
                    id_list.append(df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values[0])
                    #print(df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values[0])
                except:
                    pass
                    #print('--->', df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values)
            #print()
        dfm = pd.DataFrame({'images':id_list})
        dfm['task'] = task
        dfm['user'] = user
        dfs.append(dfm)
df_missing = pd.concat(dfs)
#df_missing.to_excel(f'{datapath}/missing.xlsx')
df_missing.head()       

,images,task,user


#### Checking duplicates

In [15]:
duplicates = df_labeling.loc[df_labeling.duplicated(['task', 'user', 'image name', 'id_image'], keep=False), ['task', 'user', 'image name', 'id_image', 'class']]
duplicates.sort_values(by=['user', 'image name', 'task'], inplace=True)
#duplicates.to_excel(f'{datapath}/duplicates.xlsx')
duplicates.head(30)

,task,user,image name,id_image,class


### Analysing the [inter-annotator agreement](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) on the results

In [16]:
l1 = df_labeling.user.unique()
iter_users = list(itertools.product(l1,l1))
df_iaa = pd.DataFrame(index=l1, columns=l1)

In [17]:
for task in df_labeling.task.unique():
    for user1,user2 in iter_users:
        classesA = df_labeling.loc[(df_labeling.user == user1) & (df_labeling.task == task),['id_image', 'class']]
        classesA.sort_values(by=['id_image'], inplace=True)

        classesB = df_labeling.loc[(df_labeling.user == user2) & (df_labeling.task == task),['id_image', 'class']]
        classesB.sort_values(by=['id_image'], inplace=True)
        
        classesAB = pd.merge(classesA, classesB, on=['id_image'])
        classesAB.drop_duplicates(subset='id_image', keep = 'first', inplace=True) 
        classesAB.drop('id_image', axis=1, inplace=True)
        classesAB.dropna(inplace=True)

        agreement = cks(classesAB['class_x'], classesAB['class_y'])
        df_iaa.loc[user1,user2] = f'{agreement:.3f}/({len(classesAB)})'  
        df_iaa.index.name = f'Task_{task}'
    print(tabulate(df_iaa, headers='keys', tablefmt='psql'))
    print()

+----------+--------------+--------------+--------------+--------------+--------------+
| Task_4   | A001         | A002         | A004         | A005         | A006         |
|----------+--------------+--------------+--------------+--------------+--------------|
| A001     | 1.000/(1079) | 0.223/(1079) | 0.287/(1079) | 0.057/(1079) | 0.293/(1079) |
| A002     | 0.223/(1079) | 1.000/(1079) | 0.245/(1079) | 0.090/(1079) | 0.163/(1079) |
| A004     | 0.287/(1079) | 0.245/(1079) | 1.000/(1079) | 0.133/(1079) | 0.200/(1079) |
| A005     | 0.057/(1079) | 0.090/(1079) | 0.133/(1079) | 1.000/(1079) | 0.085/(1079) |
| A006     | 0.293/(1079) | 0.163/(1079) | 0.200/(1079) | 0.085/(1079) | 1.000/(1079) |
+----------+--------------+--------------+--------------+--------------+--------------+



In [18]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5395 entries, 0 to 1078
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   task               5395 non-null   int64         
 1   user               5395 non-null   object        
 2   date               5395 non-null   datetime64[ns]
 3   image name         5395 non-null   object        
 4   id_image           5395 non-null   int16         
 5   class              5395 non-null   object        
 6   image_appearances  5395 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 305.6+ KB


In [37]:
df_labeling['round'] = 2
df_labeling.drop('image_appearances', inplace=True, axis=1)

In [18]:
df_labeling.to_hdf('../data/df_labeling.hdf', key='round2')